In [ ]:
from create_epochs import *
import matplotlib.pyplot as plt
import mne
import pandas as pd
import os
from copy import deepcopy
from collections import namedtuple
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# If your OS is a UNIX distribution (Linux, macOS), change '\\' to '/'
# (I don't know why on Windows, paths have to be escaped)
eeglab_raw = mne.io.read_raw_eeglab('files_to_read\\S1.set')
eeglab_epochs = mne.read_epochs_eeglab('files_to_read\\S1_elist_HW3bins_be.set')

# This function also works for when eeglab_epochs has been ADed
epochs = epoching(eeglab_raw, eeglab_epochs,
                  'files_to_read\\HW3-tutorial-bdf.txt', 'files_to_read\\S1_elist_HW3bins.txt', tmin=-0.2)

In [ ]:
epochs.event_id

In [ ]:
bins = parse_bdf('files_to_read\\HW3-tutorial-bdf.txt')
bins

In [ ]:
for v in bins.values():
    print('\n',v)
    display(epochs[v])

In [ ]:
# Resetting channel types as HEO and VEO are read as 'eeg' by MNE
mapping = {}
for ch in epochs.ch_names:
    mapping[ch] = 'eeg' if ch != 'HEO' and ch != 'VEO' else 'eog'

epochs = epochs.set_channel_types(mapping=mapping)

In [ ]:
# However, excluding HEO and VEO from re-referencing will result in the returned Epochs object loosing two channels
# This can be solved by later adding HEO and VEO back (not the focus of this assignment anyway)
epochs_m1m2 = deepcopy(epochs).set_eeg_reference(ref_channels=['M1','M2'], ch_type='eeg')
epochs_fzczpz = deepcopy(epochs).set_eeg_reference(ref_channels=['FZ','CZ','PZ'], ch_type='eeg')
epochs_avg = deepcopy(epochs).set_eeg_reference(ref_channels=[i for i in epochs.ch_names if i!='HEO' and i!='VEO'],
                                             ch_type='eeg')

In [ ]:
'''
In the cell above, epochs.copy().set_eeg_reference() works as well, but personally I like copy.deepcopy()
to be safer (?), though I am unsure if using deep copies more than necessary is the best practice in Python.
If you don't copy it, however, guess what happens?

Also, coming from C++, it took me a while to understand Python shallow and deep copies as Python is a rather
unorthodox (?) language for its "pass-by-assignment" syntax and mutable/immutable objects. (Okay, yes, there's
this "mutable" keyword in C++ — I'll just admit I almost flunked C++ lmfao ><)
'''

a = [[0],[1],[2]]
b = a
print(f'id(a) = {id(a)}; id(b) = {id(b)}')
a.append([3])
print(f'  a = {a}; b = {b}', '\n')  # This is why I am against people saying "Python is easy"...

c = [[0],[1],[2]]
d = c.copy()
print(f'id(c) = {id(c)}; id(d) = {id(d)}')
c.append([3])
print(f'  c = {c}; d = {d}')
c[0][0] = 'O'
print(f'  c = {c}; d = {d}', '\n')

e = [[0],[1],[2]]
f = deepcopy(e)
print(f'id(e) = {id(e)}; id(f) = {id(f)}')
e.append([3])
print(f'  e = {e}; f = {f}')
e[0][0] = 'O'
print(f'  e = {e}; f = {f}')

In [ ]:
# Reading EEGLAB-rereferenced datasets
eeglab_epochs_m1m2 = mne.read_epochs_eeglab('files_to_read\\S1_elist_HW3bins_be_chop.set')
eeglab_epochs_fzczpz = mne.read_epochs_eeglab('files_to_read\\S1_elist_HW3bins_be_chop-FZCZPZ.set')
eeglab_epochs_avg = mne.read_epochs_eeglab('files_to_read\\S1_elist_HW3bins_be_chop-avg.set')

In [ ]:
# Checking if they are identical as far as rounding errors are concerned
print(abs(epochs_m1m2.copy().get_data(units={'eeg':'uV','eog':'uV'}) - eeglab_epochs_m1m2.copy().get_data(units='uV')).max())
print(abs(epochs_fzczpz.copy().get_data(units={'eeg':'uV','eog':'uV'}) - eeglab_epochs_fzczpz.copy().get_data(units='uV')).max())
print(abs(epochs_avg.copy().get_data(units={'eeg':'uV','eog':'uV'}) - eeglab_epochs_avg.copy().get_data(units='uV')).max())

In [ ]:
# Okay well, on second thought, it's really not useful to create  a named tuple in this particular case LOL
# The code in the following cells only need them to be a regular tuple
My_Data = namedtuple('My_Data',['M1M2','FZCZPZ','Avg'])
my_epochs, my_data = [epochs_m1m2, epochs_fzczpz, epochs_avg], [None,None,None]

for i, x in enumerate(my_epochs):
    evokeds = [x[v].average() for v in bins.values()]
    n_bins = len(evokeds)
    n_chs, n_times = evokeds[0].get_data().shape
    data = np.array([evokeds[i].get_data(units='uV') for i in range(len(evokeds))], dtype=object).reshape(n_bins, n_chs, n_times)
    my_data[i] = data

my_data = My_Data(my_data[0],my_data[1],my_data[2])

In [ ]:
# Creating a bins dictionary to more conveniently access bin idices through bin names
bins_dict = {}
for k in bins.values():
    bins_dict[k] = list(bins.values()).index(k)

bins_dict

In [ ]:
# Export rereferenced data from EEGLAB: ERPLAB > Export & Import ERP > Export ERP to text (universal)
os.chdir('exported_data_txt')
fnames = ['m1m2_Class_G .txt', 'm1m2_Class_UG.txt',
          'fzczpz_Class_G .txt', 'fzczpz_Class_UG.txt',
          'avg_Class_G .txt', 'avg_Class_UG.txt']

# Checking if Python data arrays and EEGLAB data arrays are identical as far as rounding errors are concerned
for i, f in enumerate(fnames):
    df = pd.read_csv(f, sep='	').drop(columns=['time','Unnamed: 35','HEO','VEO'])
    #display(df)
    eeglab_data = df.to_numpy().transpose()

    try:
        print(abs(my_data[int(i/2)][bins_dict['/'.join(f[-12:-5].split('_'))]]-eeglab_data).max())
    except KeyError:
        print(abs(my_data[int(i/2)][bins_dict['/'.join(f[-12:-4].split('_'))]]-eeglab_data).max())

os.chdir('..')

In [ ]:
ch_dict, ref_dict = {}, {'Ref = (M1+M2)/2':0, 'Ref = (FZ+CZ+PZ)/3':1, 'Ref = avgchan(1:32)':2}
for i, x in enumerate(epochs.ch_names):
    ch_dict[x] = i
    
ch_to_plot = ['FZ', 'CZ', 'PZ']
bins_to_plot = ['Class/G', 'Class/UG']

t = [i for i in range(-200, 1200)]
linestyles = ['solid', 'solid']
colors = ['black','red']

for k, v in ref_dict.items():
    figure, axes = plt.subplots(1,3, figsize=(20, 3), sharey=True)
    for ax, ch in zip(axes.copy().flatten(), ch_to_plot):
        for i, x in enumerate(bins_to_plot):
            ax.plot(t, my_data[v][int(list(bins.keys())[int(list(bins.values()).index(x))])-1][ch_dict[ch]],
                    linestyle=linestyles[i], color=colors[i], label=x)
        ax.axvline(x=0, color='black', linewidth=0.5)
        ax.axhline(y=0, color='black', linewidth=0.5)
        ax.set_title(ch)
        ax.set_xlabel('Time (ms)')
        ax.set_xlim(-200, 1200)
        ax.set_ylabel('µV')
        ax.set_ylim(-10, 10)
        ax.invert_yaxis()
        ax.yaxis.set_tick_params(labelbottom=True)
        hdl, lbl = ax.get_legend_handles_labels()
    figure.legend(hdl, lbl, loc='upper left', bbox_to_anchor=[-0.001, 0.001])
    figure.tight_layout()
    plt.suptitle(k, y=1.1, fontsize=20)
    plt.show()